# update schemas.yml with output from trace-inputlocator script

In [59]:
import yaml
import os
import cea.inputlocator

In [60]:
# some constants
TRACE_OUTPUT_YML = r"C:\Users\darthoma\Documents\CityEnergyAnalyst\projects\reference-case-open\baseline\outputs\trace_inputlocator.output.yml"
SCENARIO = r"C:\Users\darthoma\Documents\CityEnergyAnalyst\projects\reference-case-open\baseline"
SCHEMAS_YML = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\schemas.yml"

### clean up trace output

In [61]:
with open(TRACE_OUTPUT_YML, 'r') as trace_output_fp:
    trace_output = yaml.load(trace_output_fp)

In [62]:
# this tests whether we have any non-scenario file_paths - it seems we don't. this is good!
print '\n'.join(k for k in trace_output.keys() if not os.path.commonprefix([SCENARIO, trace_output[key]["file_path"]]) == SCENARIO)

In [63]:
# convert all file paths to relative and replace backslashes with forward slashes
for key in trace_output:
    file_path = trace_output[key]["file_path"]
    file_path = os.path.normpath(file_path)
    file_path = os.path.relpath(file_path, SCENARIO)
    file_path = file_path.replace("\\", "/")
    file_path = file_path.replace("B01", "B001")
    trace_output[key]["file_path"] = file_path

In [64]:
# remove "description" filed (not used in schemas.yml)
for key in trace_output:
    if "description" in trace_output[key]:
        del trace_output[key]["description"]

###  make sure all methods in schemas.yml are present in InputLocator

In [65]:
with open(SCHEMAS_YML, 'r') as schemas_fp:
    schemas = yaml.load(schemas_fp)

In [66]:
locator = cea.inputlocator.InputLocator(SCENARIO)

In [67]:
# the output here should be empty!
for key in schemas:
    if not hasattr(locator, key):
        print "locator not found:", key

### i'm curious: what methods are not in the trace-inputlocator output?

In [68]:
# expect an empty list...
for key in schemas:
    if not key in trace_output:
        print "missing key in trace_output:", key

### assuming we're not missing any methods, replace the schemas.yml file 

In [69]:
with open(SCHEMAS_YML, 'w') as schemas_fp:
    yaml.dump(trace_output, schemas_fp)